In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from langchain_groq import ChatGroq
model = ChatGroq(model="gemma2-9b-it")
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000002104FE378E0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000002104FE61B70>, model_name='gemma2-9b-it', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [3]:
from langchain_core.messages import HumanMessage
model.invoke([
    HumanMessage(content="Hi, my name is Akshay and I am a research engineer at VizuaraAI Labs")
])

AIMessage(content="Hi Akshay, it's nice to meet you! \n\nIt's great to know you're a research engineer at VizuaraAI Labs. What kind of research are you working on? I'm always interested in learning about new developments in AI.\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 57, 'prompt_tokens': 27, 'total_tokens': 84, 'completion_time': 0.103636364, 'prompt_time': 0.003344456, 'queue_time': 0.328542942, 'total_time': 0.10698082}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--8a8e255f-c931-4f47-85c8-0e571c4076e3-0', usage_metadata={'input_tokens': 27, 'output_tokens': 57, 'total_tokens': 84})

In [4]:
from langchain_core.messages import AIMessage

model.invoke([
    HumanMessage(content="Hi, my name is Akshay and I am a research engineer at VizuaraAI Labs"),
    AIMessage(content="Hi Akshay, it's nice to meet you! \n\nIt's great to know you're a research engineer at VizuaraAI Labs. What kind of research are you working on? I'm always interested in learning about new developments in AI.\n"),
    HumanMessage(content="Hey what's my name and and I do?")
])

AIMessage(content="You said your name is Akshay and that you are a research engineer at VizuaraAI Labs.  \n\nIs there anything else you'd like to tell me about yourself or your work? I'm eager to learn more! 😊  \n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 52, 'prompt_tokens': 103, 'total_tokens': 155, 'completion_time': 0.094545455, 'prompt_time': 0.005432756, 'queue_time': 0.24804349299999998, 'total_time': 0.099978211}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--51e7eafc-5d7d-44a3-8972-a9ca92372c55-0', usage_metadata={'input_tokens': 103, 'output_tokens': 52, 'total_tokens': 155})

In [7]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store={}
def get_session_history(session_id: str)-> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory() 
    return store[session_id]

with_message_history = RunnableWithMessageHistory(model, get_session_history)

In [9]:
# config={"configurable":{"session_id": "chat1"}}

from typing import cast
from langchain_core.runnables import RunnableConfig

config = cast(RunnableConfig, {"configurable": {"session_id": "chat1"}})

In [10]:
with_message_history.invoke([
    HumanMessage(content="Hi, my name is Akshay and I am a research engineer at VizuaraAI Labs")
],
config=config
)

AIMessage(content="Hi Akshay, it's nice to meet you!\n\nAs a research engineer at VizuaraAI Labs, what kind of projects are you working on?\n\nI'm always interested in learning more about the latest advancements in AI.  Perhaps you could tell me a little about your work?\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 63, 'prompt_tokens': 27, 'total_tokens': 90, 'completion_time': 0.114545455, 'prompt_time': 0.00214298, 'queue_time': 0.24658459800000002, 'total_time': 0.116688435}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--ebfa4fe6-e260-42b9-bbb3-75361e211403-0', usage_metadata={'input_tokens': 27, 'output_tokens': 63, 'total_tokens': 90})

In [11]:
## Change the config-->Session id
config1 = cast(RunnableConfig, {"configurable": {"session_id": "chat1"}})
response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config1
)

In [13]:
response.content

'Your name is Akshay.  \n\nI remembered it from our earlier conversation! 😊  Is there anything else I can help you with?\n'

In [17]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate([
    ("system", "You are the helpful assistant. Answer all the questions to the best of your ability"),
    MessagesPlaceholder(variable_name="message")
])


In [18]:
chain = prompt | model

In [19]:
response=chain.invoke({"message":[HumanMessage(content="Hi, my name is akshay")]})
response.content

"Hello Akshay!\n\nIt's nice to meet you. What can I do for you today? 😊  \n\nI'm ready to answer your questions and help in any way I can.  \n"

In [31]:
with_message_history=RunnableWithMessageHistory(chain, get_session_history)

In [22]:
config2 = cast(RunnableConfig, {"configurable": {"session_id": "chat3"}})
response = with_message_history.invoke(
    [HumanMessage(content="Hi, my name is akshay")],
    config=config2
)

response

AIMessage(content="Hi Akshay! It's nice to meet you.  \n\nHow can I help you today? 😊  \n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 88, 'total_tokens': 113, 'completion_time': 0.045454545, 'prompt_time': 0.005442069, 'queue_time': 0.24597013099999998, 'total_time': 0.050896614}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--1a97b952-48f0-441e-9a62-26d01ca39f7e-0', usage_metadata={'input_tokens': 88, 'output_tokens': 25, 'total_tokens': 113})

In [23]:
from langchain_core.output_parsers import StrOutputParser
parser = StrOutputParser()

In [24]:
# Add some complexity
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate([
    ("system", "You are the helpful assistant. Answer all the questions to the best of your ability in {language}"),
    MessagesPlaceholder(variable_name="message")
])

chain = prompt|model|parser

In [27]:
response=chain.invoke({"message":[HumanMessage(content="Hi my nname is akshay")],"language":"marathi"})
response

'नमस्कार अक्षय! मला माहित आहे की तुमचे नाव अक्षय आहे. मी तुमच्या मदतीसाठी येथे आहे. तुम्हाला काय सांगायचे आहे?  😊\n'

In [28]:
with_message_history=RunnableWithMessageHistory(
    chain, 
    get_session_history,
    input_messages_key="message"
)

In [30]:
config3 = cast(RunnableConfig, {"configurable": {"session_id": "chat4"}})
response = with_message_history.invoke(
    {"message": [HumanMessage(content="Hi, my name is akshay")], "language": "sanskrit"},
    config=config3
)

response

'नमस्ते अक्षय! \n\nआपका स्वागत है। मेरा नाम कृत्रिम बुद्धिमत्ता है, और मैं आपकी सहायता करने के लिए यहाँ हूँ। आप मुझसे क्या जानना चाहते हैं? \n\n\n(Namaste Akshay! \n\nWelcome. My name is Artificial Intelligence, and I am here to assist you. What can I help you with?) \n\n'

In [34]:
## Mananging conversation history
from langchain_core.messages import HumanMessage,SystemMessage,AIMessage, trim_messages 

trimmer = trim_messages(
    max_tokens=70,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human"
)


messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm karan"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]

trimmer.invoke(messages)

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

d:\Langchain\.venv\lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Admin\.cache\huggingface\hub\models--gpt2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

[SystemMessage(content="you're a good assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content="hi! I'm karan", additional_kwargs={}, response_metadata={}),
 AIMessage(content='hi!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I like vanilla ice cream', additional_kwargs={}, response_metadata={}),
 AIMessage(content='nice', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='whats 2 + 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

In [39]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough

chain = (
    RunnablePassthrough.assign(messages=itemgetter("message") | trimmer)
    | prompt
    | model
)

response = chain.invoke({
    "message": messages + [HumanMessage(content="What math problem did i asked?")],
    "language": "English"
})
response.content

'You asked "whats 2 + 2" 😊  \n\n\n\nLet me know if you have any other questions!\n'

In [ ]:
# Lets wrap this message history 
with_message_history=RunnableWithMessageHistory(
    chain, 
    get_session_history,
    input_messages_key="message"
)


config4 = cast(RunnableConfig, {"configurable": {"session_id": "chat4"}})
response = with_message_history.invoke(
    {"message": [HumanMessage(content="What math problem I aksed?")], "language": "English"},
    config=config4
)

response

AIMessage(content="माफ़ करना, परंतु मुझे याद नहीं है कि आपने कौन सी गणित समस्या पूछी थी। क्या आप कृपया फिर से पूछ सकते हैं?\n\n(I'm sorry, but I don't remember what math problem you asked. Could you please ask it again?) \n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 64, 'prompt_tokens': 141, 'total_tokens': 205, 'completion_time': 0.116363636, 'prompt_time': 0.0132603, 'queue_time': 0.63044113, 'total_time': 0.129623936}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--a4c7dafb-1e7d-4a15-b788-a6c733b260eb-0', usage_metadata={'input_tokens': 141, 'output_tokens': 64, 'total_tokens': 205})